<a href="https://colab.research.google.com/github/pavansai26/titanic-data-prediction-using-server/blob/master/titanic_data_prediction_using_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [0]:
import pandas as pd
import numpy as np
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [0]:
titanic_data = pd.read_csv('titanic.csv')
titanic_data.head()

In [0]:
titanic_data.columns

In [0]:
titanic_data.isnull().sum()

In [0]:
titanic_data.info()

In [0]:
titanic_data.drop('Cabin',axis=1,inplace=True)

In [0]:
titanic_data.isnull().sum()

In [0]:
titanic_data['Age'].fillna((titanic_data['Age'].mean()),inplace=True)

In [0]:
titanic_data.isnull().sum()

In [0]:
titanic_data.dropna(inplace=True)

In [0]:
titanic_data.isnull().sum()

In [0]:
#To create dummies (categories)
sex = pd.get_dummies(titanic_data["Sex"],drop_first=True)
sex.head(2)

In [0]:
Pcl = pd.get_dummies(titanic_data["Pclass"],drop_first=True)
Pcl.head(2)

In [0]:
embark = pd.get_dummies(titanic_data["Embarked"],drop_first=True)
embark.head(2)

In [0]:
titanic_data = pd.concat([titanic_data,sex,Pcl,embark],axis = 1)
titanic_data.head(2)

In [0]:
titanic_data.drop(['Sex','Embarked','Pclass','Name','Ticket','PassengerId'],axis=1,
                  inplace=True)
titanic_data.head()

In [0]:
#Training and Testing data
X = titanic_data.drop("Survived",axis=1)
y = titanic_data["Survived"]
from sklearn.model_selection import train_test_split

In [0]:
#Splitting into training set and testing set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [0]:
predictions = logmodel.predict(X_test)

In [0]:
#Metrics
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions)*100)

In [0]:
predictions = logmodel.predict([[22.0,1,0,7.2500,1,0,1,0,1]])
predictions

In [0]:
!pip install flask-ngrok

In [0]:
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify
app =Flask(__name__)
run_with_ngrok(app) #starts ngrok when the app is running
@app.route("/<int:Age>/<int:SibSp>/<int:Parch>/<float:Fare>/<Gender>/<int:PClass>/<Place>")
def home(Age,SibSp,Parch,Fare,Gender,PClass,Place):
  p = []
  p += [Age,SibSp,Parch,Fare]
  if Gender.casefold() == "m":
    p+=[1]
  else:
    p+=[0]
  if PClass == 2:
    p+=[1,0]
  elif PClass == 3:
    p+=[0,1]
  else:
    p+=[0,0]
  if Place.casefold() == "queenstown":
    p+=[1,0]
  elif Place.casefold() == "southampton":
    p+=[0,1]
  else:
    p+=[0,0]
  arr = np.array([p])
  predict =logmodel.predict(arr)
  if predict == [1]:
    result = {'result':'Survived'}
  else:
    result = {'result':'NotSurvived'}
  
  return jsonify(result)
app.run()